In [ ]:
import re
import csv
import json
import ast

#https://sparkbyexamples.com/pandas/pandas-filter-rows-by-conditions/#:~:text=You%20can%20use%20df%5Bdf,new%20DataFrame%20with%20selected%20rows.&text=You%20can%20also%20write%20the%20above%20statement%20with%20a%20variable.

def condition(id):
    #keywords, crew, cast, tagline, prodution_companies,genres, overview
    #note: tagline and overfiew are filtered with dropna()
    #this is needed now
    length = len(complete[complete["id"] == id]["keywords"].to_string())
    if(complete[complete["id"] == id]["keywords"].to_string()[length-2:]=="[]"):
        return False
    length = len(complete[complete["id"] == id]["cast"].to_string())
    if(complete[complete["id"] == id]["cast"].to_string()[length-2:]=="[]"):
        return False
    length = len(complete[complete["id"] == id]["production_companies"].to_string())
    if(complete[complete["id"] == id]["production_companies"].to_string()[length-2:]=="[]"):
        return False
    length = len(complete[complete["id"] == id]["genres"].to_string())
    if(complete[complete["id"] == id]["genres"].to_string()[length-2:]=="[]"):
        return False
    length = len(complete[complete["id"] == id]["tagline"].to_string())
    if(complete[complete["id"] == id]["tagline"].to_string()[length-4:]=="<NA>"):
        return False
    length = len(complete[complete["id"] == id]["overview"].to_string())
    if(complete[complete["id"] == id]["overview"].to_string()[length-4:]=="<NA>"):
        return False
    
    return True



def populateNames(id, string):
    dictTemp  = complete[complete["id"] == id][string].to_dict()
    jsons = dictTemp[list(dictTemp.keys())[0]][1:-1].split("}, ")

    #print(jsons)
    names = ""
    cnt = 0
    for item in jsons:
        if(cnt == len(jsons)-1):
            tempDict = ast.literal_eval(item)
            names+=str(tempDict["name"])
        else:
            tempDict = ast.literal_eval(item+"}")
            names+=str(str(tempDict["name"])+" ")
        cnt += 1

    #print(names)
    return names


def provideData2(userId, id,  rating):
    movieData = []
    movieData.append(userId)
    movieData.append(int(id))
    movieData.append(float(rating))  
    movieData.append(populateNames(id, "keywords"))
    movieData.append(populateNames(id, "cast"))
    movieData.append(populateNames(id, "production_companies"))
    movieData.append(populateNames(id, "genres"))
    movieData.append(complete[complete["id"] == id]["tagline"].to_string().split("    ")[1])
    movieData.append(complete[complete["id"] == id]["overview"].to_string().split("    ")[1])
    return movieData



#https://www.kaggle.com/code/fabiendaniel/film-recommendation-engine/notebook

def provideData1(userId, id,  rating):
    movieData = []
    movieData.append(userId)
    movieData.append(int(id))
    movieData.append(float(rating))

    regex = r"'name': '[\sa-zA-Z]*'}";    
    matches  = re.findall(regex, complete[complete["id"] == id]["keywords"].to_string())
    movieData.append(' '.join([item[9:-2] for item in matches]))
    matches  = re.findall(regex, complete[complete["id"] == id]["genres"].to_string())
    movieData.append(' '.join([item[9:-2] for item in matches]))

    regex = r"'name': '[\sa-zA-Z]*',";
    matches  = re.findall(regex, complete[complete["id"] == id]["cast"].to_string())
    movieData.append(' '.join([item[9:-2] for item in matches]))
    matches  = re.findall(regex, complete[complete["id"] == id]["production_companies"].to_string())
    movieData.append(' '.join([item[9:-2] for item in matches]))


    movieData.append(complete[complete["id"] == id]["tagline"].split("    ")[1])
    movieData.append(complete[complete["id"] == id]["overview"].split("    ")[1])

    return movieData



selectedFeatures = []

#number of users to consider
for i in range(1, 20):

    userRatings = complete[complete["userId"] ==str(i)]
    filtered = []


    for id,rating in zip(list(userRatings["id"]), list(userRatings["rating"])):
        if (condition(id)):
            # provideData2(i, id, rating)
            movieData  = provideData2(i, id, rating)
            filtered.append(movieData)


    #minimum number of ratings enforced here
    if(len(filtered) >= 10):
        selectedFeatures.extend(filtered)